# Run the Tracker

This jupyter notebook bundles all necessary high-level steps to train the tracker. The first step is to include all necessary modules.

In [1]:
from tqdm import tqdm
import pickle
import sys
import numpy as np
from datetime import datetime
import os
sys.path.append('build/')
from cpp_utils import Simulation
from time import time, sleep
from python.plotter import Plotter
from python.constants import *
from python.accurateTracker import AccurateTracker
from python.confidentTracker import ConfidentTracker
from python.metrics import Confidence, Accuracy

A simple 2D simulation simulates is used to generate the data of the robot perceptions. The next cells runs a new simulation to generate fresh data. The fresh data is saved by a simulation log pickle file. This cell is optional, we can also load the data from an old simulation log.

In [2]:
T_simulation = 5 * 60 # 5 minutes

sim = Simulation(
    T_step=0.1, 
    N_humans=3, 
    N_robots=1
)
sim_states = []
N_minutes = int(T_simulation / 60)
N_hours = int(T_simulation / 3600)
pbar = tqdm(range(0, N_minutes), desc='Simulation')
for i in pbar:
    sim_states += sim.step(int(60 / sim.T_step))
    pbar.set_postfix(
    {'Simulated time': '{:d}:{:02d} of {:d}:{:02d} hours'.format(
        int(i / 60), 
        i % 60, 
        int(T_simulation / 3600), 
        int(T_simulation / 60) % 60
    )})

sim_log = {
        'T_step': sim.T_step,
        'T_simulation': T_simulation,
        'N_humans': sim.N_humans,
        'N_robots': sim.N_robots,
        'sim_states': sim_states,
    }
filename = os.path.join(LOG_FOLDER, 'log_' + datetime.now().strftime("%Y-%m-%d_%H-%M-%S") + '.pkl')
with open(filename, 'wb') as outp:
    pickle.dump(sim_log, outp, pickle.HIGHEST_PROTOCOL)
    

Simulation: 100%|██████████| 5/5 [00:00<00:00, 14.84it/s, Simulated time=0:04 of 0:05 hours]


This cell loads the simulation data from a simulation log. If no fresh data is available, an old simulation log is loaded.

In [3]:
if "sim_log" not in locals():
    filename = os.path.join(LOG_FOLDER, 'log_2024-08-30_10-47-52.pkl')
    with open(filename, 'rb') as f:
        sim_log = pickle.load(f)
sim_states = sim_log['sim_states']
T_simulation = sim_log['T_simulation']
T_step = sim_log['T_step']
N_humans = sim_log['N_humans']
N_robots = sim_log['N_robots']
# load simulation to have access to member utility functions
sim = Simulation(
    T_step=0.1, 
    N_humans=3, 
    N_robots=1
)

With the simulation data loaded, the actual tracker can be loaded. The tracker is now simulated in another loop over the simulation time. The robot perceptions are just playback from the simulation log. Different Trackers can be tested by creating the tracker object from different classes.

In [9]:
plot = False
record_video = False # slows down loop massively!

confidentTracker = ConfidentTracker(N_robots=N_robots, include_observations=True)
accurateTracker = AccurateTracker(N_robots=N_robots, include_observations=True, train=False)

if plot: plotter = Plotter(record_frames=record_video)

pbar = tqdm(range(0, len(sim_states)), desc='Simulation')

simulation_time = 0
confidentTracker_node_probabilities = []
accurateTracker_node_probabilities = []
for i in pbar:
    sim_state = sim_states[i]

    # outer list: robots, inner list: perceived humans for every robot
    robot_perceptions = [{ 
        "ego_position": agent['ego_position'],
        "observable_nodes": agent['observable_nodes'],
        "perceived_humans": agent['perceived_humans'],
    } for agent in sim_state if agent['type'] == 'robot']

    confidentTracker_node_probabilities.append(confidentTracker.add_observation(robot_perceptions))
    _ = confidentTracker.predict()

    accurateTracker_node_probabilities.append(accurateTracker.add_observation(robot_perceptions))
    _ = accurateTracker.predict()

    if plot: plotter.update(sim_state, confidentTracker_node_probabilities[-1])    

    pbar.set_postfix(
        {'Simulated time': '{:d}:{:02d} of {:d}:{:02d} hours'.format(
            int(simulation_time / 3600), 
            int(simulation_time / 60) % 60, 
            int(T_simulation / 3600), 
            int(T_simulation / 60) % 60
        )})
    
    simulation_time += T_step

if record_video: plotter.create_video(T_step)

Simulation: 100%|██████████| 3000/3000 [00:04<00:00, 743.71it/s, Simulated time=0:04 of 0:05 hours]


The predicted node probabilites for every node (the output of the tracker) is added to the simulation log and the updated simulation log is saved again.

In [5]:
# sim_log["node_probabilities"] = node_probabilities
# with open(filename, 'wb') as outp:
#     pickle.dump(sim_log, outp, pickle.HIGHEST_PROTOCOL)

# accurateTracker.save_trained_model()

Now the performance of the tracker is evaluated using two performance metrics: Confidence and Accuracy. Both are defined as loss metrics, hence 0 is the optimal value and the larger the value is, the worse the metric is.

In [11]:
confidentTracker_confidence = Confidence(sim_log, confidentTracker_node_probabilities).per_graph(only_count_nodes_with_no_humans=False)
accurateTracker_confidence = Confidence(sim_log, accurateTracker_node_probabilities).per_graph(only_count_nodes_with_no_humans=False)
print("Confidence of ConfidentTracker:", confidentTracker_confidence, 
      "Confidence of AccurateTracker:", accurateTracker_confidence)
confidentTracker_accuracy = Accuracy(sim_log, confidentTracker_node_probabilities).per_graph()
accurateTracker_accuracy = Accuracy(sim_log, accurateTracker_node_probabilities).per_graph()
print("Accuracy of ConfidentTracker:", confidentTracker_accuracy, 
      "Accuracy of AccurateTracker:", accurateTracker_accuracy)
print(accurateTracker.probabilities)

Confidence of ConfidentTracker: 0.025406383999070803 Confidence of AccurateTracker: 0.05864315802496278
Accuracy of ConfidentTracker: 0.004432114637548517 Accuracy of AccurateTracker: 0.005581027963325148
[3.33333333e-04 1.00000000e+00 7.28333333e-02 2.58333333e-02
 7.00000000e-03 1.45333333e-01 4.13333333e-02 1.88333333e-02
 0.00000000e+00 2.71666667e-02 8.33333333e-03 0.00000000e+00
 1.21666667e-02 1.25833333e-01 3.88333333e-02 2.66666667e-02
 5.48333333e-02 3.38333333e-02 1.16666667e-03 0.00000000e+00
 1.27500000e-01 1.48333333e-02 9.83333333e-03 0.00000000e+00
 6.25000000e-02 5.66666667e-03]
